In [2]:
import pandas as pd
import numpy as np

In [23]:
# TODO: 
# 1. Choose better negative sample (ppl with no complications)
# 2. Try different complications and subset broader
# 3. Subset patients with less than 4 visits
# 4. Preprocess Lab results earlier

In [28]:
Lab = pd.read_csv("FullData/LabReduced.csv")
Exam = pd.read_csv("FullData/Exam.csv", usecols = ['Patient_ID', 'Exam1', 'Result1_calc', 'Exam2', 'Result2_calc', 'UnitOfMeasure_calc', 'DateCreated'])
DiseaseCase = pd.read_csv("FullData/DiseaseCase.csv")

In [29]:
# Moved coercion of Test results to floats up here from combined cell
print(Lab['TestResult_calc'].isna().sum())
Lab['TestResult_calc'] = pd.to_numeric(Lab.TestResult_calc,errors='coerce')
print(Lab['TestResult_calc'].isna().sum())
Lab = Lab.dropna(subset=['TestResult_calc'])
print(Lab['TestResult_calc'].isna().sum())
print(len(Lab.index))

0
174177
0
25163370


In [ ]:
## FILTER lab with better negative sample

In [ ]:
# Encode lab results
for index, row in Results.iterrows() :
    test = row['Name_calc']
    value = row['TestResult_calc']
    #feature = [0,0,0]
    t1 = float(turtiles.loc[turtiles['TestName'] == test]['t1'])
    t2 = float(turtiles.loc[turtiles['TestName'] == test]['t2'])
    #print(t1,t2,value)
    if value <= t1 : ## Changed from < to <=
        feature = 1.0 #[1,0,0]
    elif value >= t2 : ## Changed from > to >=
        feature = 3.0 #[0,0,1]
    else :
        feature = 2.0 #[0,1,0]
    Results.at[index, 'TestResult_calc'] = feature
    
        

In [ ]:
## NEW Moved up
Results = Results.groupby('Patient_ID').apply(lambda g: g.sort_values(by = 'PerformedDate', ascending =True))
Results = Results.reset_index(drop = True)
Results.head()

In [ ]:
## NEW
rem = []
for index, row in Results.iterrows() :
    temp = row[['Patient_ID', 'Name_calc','TestResult_calc']]
    if index == 0 :
        prev = temp
        continue
    if temp.equals(prev):
        rem.append(index)
    prev = temp
    #print(prev)

print(len(rem))


In [ ]:
## NEW
Results_u = Results
print(len(Results_u.index))
Results_u = Results_u.drop(rem)
print(len(Results_u.index))

In [30]:
# DiseaseCase
DiseaseCase['Disease'] = DiseaseCase['Disease'].replace("Diabetes Mellitus (ml)","Diabetes Mellitus") # Make all DM2 diagnosis' the same
DiseaseCase['DateOfOnset'] = pd.to_datetime(DiseaseCase.DateOfOnset,format='%Y-%m-%d')
DiseaseCase = DiseaseCase.sort_values(by='DateOfOnset', ascending = True)
DiseaseCase = DiseaseCase.drop_duplicates(subset = ['Patient_ID'])

In [31]:
Exam1 = Exam.drop(columns=['Exam2','Result2_calc']).rename(columns={'Exam1':'Name_calc','Result1_calc':'TestResult_calc'})
Exam2 = Exam.drop(columns=['Exam1','Result1_calc']).rename(columns={'Exam2':'Name_calc','Result2_calc':'TestResult_calc'})
Exam = pd.concat([Exam1,Exam2]).dropna()
print(Exam['Name_calc'].value_counts(ascending=False))

sBP (mmHg)                  2570192
dBP (mmHg)                  2570192
Weight (kg)                 1102589
BMI (kg/m^2)                 910459
Height (cm)                  736942
Waist Circumference (cm)     115353
PEFR (L/min)                     35
Name: Name_calc, dtype: int64


In [32]:
tests = ['Alanine Aminotransferase in Serum or Plasma (ALT)',
              'Aspartate Aminotransferase in Serum or Plasma (AST)',
              'Urea (mM)',
              'Plasma Creatinine Clearance (eGFR)',
              'Potassium (mM)',
              'Sodium (mM)',
              'Gamma Glutamyl Transferase in Serum or Plasma (GGT)',
              'Leukocytes in Blood (WBC)',
              'Protein, total (g/L)',
              'Total Cholesterol (TCh)',
              'Total Bilirubin (TBIL)',
              'Platelets in Blood (PLT)',
              'Fasting Blood Glucose (FBG)',
              'Hemoglobin (Hb)',
              'Erythrocytes in Blood (RBC)',
              'Albumin (g/L)',
              'Hemoglobin A1c (HbA1c)',
              'BMI (kg/m^2)']
Lab = Lab.loc[Lab.Name_calc.isin(tests)]
Exam = Exam.loc[Exam.Name_calc.isin(tests)]

In [33]:
Exam = Exam.rename(columns={'DateCreated':'PerformedDate'})
Exam = Exam[Exam.columns[[0,4,1,2,3]]]
Exam = Exam.drop_duplicates(subset=['Patient_ID','PerformedDate'])

In [34]:
lab = pd.concat([Lab,Exam])
lab = lab.dropna(subset = ['PerformedDate'])


In [35]:
disease_periods = pd.read_csv('AP_posnegIDS.csv', index_col=None)
disease_periods = disease_periods.drop(disease_periods.columns[0], axis=1)
neg_sample = disease_periods.iloc[1902:3804]
disease_periods = disease_periods.iloc[0:1902]

In [36]:
patient_records = lab.loc[lab['Patient_ID'].isin(disease_periods['Patient_ID'])]
patient_records = pd.merge(patient_records, disease_periods, on='Patient_ID')
print(len(patient_records.index),patient_records.columns)

240591 Index(['Patient_ID', 'PerformedDate', 'Name_calc', 'Code_calc',
       'TestResult_calc', 'UnitOfMeasure_calc', 'DateOfOnset', 'CompOnset'],
      dtype='object')


In [37]:
relVisits.PerformedDate = pd.to_datetime(relVisits.PerformedDate,format='%Y-%m-%d')
relVisits.CompOnset = pd.to_datetime(relVisits.CompOnset,format='%Y-%m-%d')
relVisits.DateOfOnset = pd.to_datetime(relVisits.DateOfOnset,format='%Y-%m-%d')

In [38]:
relVisits = patient_records[(patient_records['PerformedDate'] < patient_records['CompOnset']) & (patient_records['PerformedDate'] >= patient_records['DateOfOnset'])]
len(relVisits.index)

76137

In [47]:
test = relVisits[['Patient_ID','PerformedDate']].groupby('Patient_ID').nunique() 
test = test[(test['PerformedDate'] < 51) & (test['PerformedDate'] > 3)]
test

,PerformedDate
Patient_ID,
1000100003603,8
1000100003646,5
1000100003708,9
1000100004129,33
1000100005507,4
...,...
12120070011568,11
12120070011754,5
12120120003869,8


In [50]:
# Count number of visits per patient
test = relVisits[['Patient_ID','PerformedDate']].groupby('Patient_ID').nunique() 
good = test[(test['PerformedDate'] < 51) & (test['PerformedDate'] > 3)]

n = len(good.index)
print(n)
relVisits = relVisits[relVisits['Patient_ID'].isin(good.index)]
print(len(relVisits.index) / n) # Average visits per person

1281
49.92818110850898


In [ ]:
# Disease Period
relVisits['dp'] = relVisits['CompOnset'] - relVisits['DateOfOnset']
dp = relVisits['dp'].mean()
print(dp)

In [ ]:
# Negative sample
from random import sample
rest = Lab[['Patient_ID','PerformedDate']].groupby('Patient_ID').nunique() < 51
rest = rest[['Patient_ID','PerformedDate']].groupby('Patient_ID').nunique() > 3
negGood = rest.index[rest['PerformedDate'] == True].tolist()
neg_ids = list(set(negGood) - set(n))
neg_samp = sample(neg_ids, len(n))
neg_rec = lab.loc[lab['Patient_ID'].isin(neg_samp)]
neg_rec = pd.merge(neg_rec, DiseaseCase[['Patient_ID','DateOfOnset']], on='Patient_ID')
print(len(neg_rec.index)/ len(neg_samp))
neg_rec = neg_rec[neg_rec['PerformedDate'] >= neg_rec['DateOfOnset']]
print(len(neg_rec.index)/ len(neg_samp))
neg_rec = neg_rec[(neg_rec['PerformedDate'] >= neg_rec['DateOfOnset']) & (neg_rec['PerformedDate'] < neg_rec['DateOfOnset']+dp) ]
print(len(neg_rec.index)/ len(neg_samp))

In [ ]:
# Append
print(len(relVisits.index))
print(len(neg_rec.index))
combined = relVisits.append(neg_rec, ignore_index=True)
combined = combined.drop(['CompOnset','dp'],axis='columns')
print(len(relVisits.index))
print(relVisits.isna().sum())
print(relVisits.dtypes)

In [ ]:
#remove vast minority units (mg/L for Total Bilirubin (n = 40) and mmol/L for Hemoglobin (n = 10))
badIndices = combined.loc[(combined.Name_calc == 'Total Bilirubin (TBIL)') & (relVisits.UnitOfMeasure_calc == 'mg/L')].index
badIndices2 = combined.loc[(combined.Name_calc == 'Hemoglobin (Hb)') & (relVisits.UnitOfMeasure_calc == 'mmol/L')].index
combined = combined.drop(badIndices)
combined = combined.drop(badIndices2)

In [ ]:
Results = combined
print(len(Results.index))

In [51]:
## LEFT OFF:
# Just moved code to subest visits to the top
# Next TODO #2, then reduce data to make subsetting quicker i.e. loc lab stuff for positive first, then sample neg ...
# ... patients > n, subset, then sample again for neg patients = n ...
# ... Probably wanna put some stuff in functions (like subsetting code chunks)